In [2]:
!pip3 install kwant

     |████████████████████████████████| 1.6 MB 4.8 MB/s 
  Created wheel for kwant: filename=kwant-1.4.3-cp37-cp37m-linux_x86_64.whl size=3690977 sha256=6a64f861742f2023374602e94f89d0b818d757698b572d1c8fcbef09b332696a
  Stored in directory: /root/.cache/pip/wheels/d6/2d/93/6f395cd3f0798d7d9e161dce1c8b8c8bbd04d4547763c926de
  Created wheel for tinyarray: filename=tinyarray-1.2.4-cp37-cp37m-linux_x86_64.whl size=227942 sha256=b2d7fca305b8721eac919e8f5044a4b9635def33e3e28781794283a8ef90aff2
  Stored in directory: /root/.cache/pip/wheels/85/c6/1c/6939e2931cfbff5df75758a06084bf67171b640ebdf5febd35
Successfully built kwant tinyarray


In [29]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.constants
import tinyarray as ta

import kwant
import sys
sys.path.append('./modules/')
#import wraparound
 
# -------------------------------------------------------------------
# Constants
qe = sp.constants.value("elementary charge") # unit: C
me = sp.constants.value("electron mass")/qe*1e-18 #unit: eV*s^2/nm^2
hP = sp.constants.value("Planck constant in eV s") #unit: eV*s
hbar = hP/(2*sp.pi) #unit: eV*s
# -------------------------------------------------------------------
 
a = 0.03
b = 0.03
V0 = 0.0
 
W = 6
L = 20
R = np.array([[np.sqrt(3)/2, 1/2],[-1/2, np.sqrt(3)/2]])
t = hbar**2/(2*me*a**2) # units: eV
#****************************************** 
tri = ta.array(((1, 0), (-0.5, 0.5 * np.sqrt(3))))
lat = kwant.lattice.Polyatomic(a * tri, ((0, 0), (0, -a / np.sqrt(3))))
#lat.a, lat.b = lat.sublattices
    
#R = np.array([[np.sqrt(3)/2, 1/2],[-1/2, np.sqrt(3)/2]])
    
#v_1 = [3*a/2, np.sqrt(3)*a/2]
#v_2 = [3*a/2, -np.sqrt(3)*a/2]

r_1 = [0,0]
r_2 = [a,0]

#lat = kwant.lattice.Polyatomic([np.dot(R,v_1), np.dot(R,v_2)], [np.dot(R,r_1), np.dot(R,r_2)])
lat.a, lat.b = lat.sublattices


#***********************************************************

# Infinite potential plane in y direction
#sys = kwant.Builder(kwant.TranslationalSymmetry(lat.vec((0, W))))
#sys[(lat.a(i,j) for i in range(L) for j in range(W))] = lambda p: 4*t
#sys[(lat.b(i,j) for i in range(L) for j in range(W))] = lambda p: 4*t
#sys[lat.neighbors(1)] = -t
 
#sys = kwant.wraparound.wraparound(sys)
 
#lead = kwant.Builder(kwant.TranslationalSymmetry((a, 0), lat.vec((0, W))))
#lead[(lat.a(0,j) for j in range(W))] = 4*t
#lead[(lat.b(0,j) for j in range(W))] = 4*t
#lead[lat.neighbors(1)] = -t
 
#lead = kwant.wraparound.wraparound(lead, keep=0)
 
#sys.attach_lead(lead)
#sys.attach_lead(lead.reversed())

#leadf=lead.finalized()

t_1=1.0 
t_2=1.0
syst = kwant.Builder()
onsite = 0
# Onsites
syst[(lat.a(n, m) for n in range(L) for m in range(L))] = onsite
syst[(lat.b(n, m) for n in range(L) for m in range(L))] = onsite
  # Hopping
hoppings = (((0, a), lat.a, lat.b), 
              ((a, 0), lat.a, lat.b), 
              ((-a/2, -np.sqrt(3)*a/2), lat.a, lat.b),
              ((-a/2, np.sqrt(3)*a/2), lat.a, lat.b))
syst[[kwant.builder.HoppingKind(*hopping) for hopping in hoppings]] = t_1
hoppings2_a = (((0, a), lat.a, lat.a), ((a, 0), lat.a, lat.a), ((a, -a), lat.a, lat.a))
#hoppings2_a = (((a/2, 3*a/2), lat.a, lat.a))
hoppings2_b = (((0, a), lat.b, lat.b), ((a, 0), lat.b, lat.b), ((a, -a), lat.b, lat.b))
#syst[[kwant.builder.HoppingKind(*hopping) for hopping in hoppings2_a]] = t_2
#syst[[kwant.builder.HoppingKind(*hopping) for hopping in hoppings2_b]] = t_2

  
LEADS = True
if LEADS:
      # Right lead
  sym_right_lead = kwant.TranslationalSymmetry(np.dot(R,[-3*a/2, -np.sqrt(3)*a/2]))
      #sym_right_lead = kwant.TranslationalSymmetry([2*a,0])
  right_lead = kwant.Builder(sym_right_lead)
  right_lead[(lat.a(n, m) for n in range(L) for m in range(L))] = onsite
  right_lead[(lat.b(n, m) for n in range(L) for m in range(L))] = onsite
    
  # Hopping
  right_lead[[kwant.
  builder.HoppingKind(*hopping) for hopping in hoppings]] = t_1
  right_lead[[kwant.builder.HoppingKind(*hopping) for hopping in hoppings2_a]] = t_2
  right_lead[[kwant.builder.HoppingKind(*hopping) for hopping in hoppings2_b]] = t_2
  
  syst.attach_lead(right_lead)
  right_lead_fin = right_lead.finalized()
  
  syst.attach_lead(right_lead.reversed())
  syst_fin = syst.finalized()
  
if(LEADS):
    return syst_fin, right_lead_fin
else:
    return syst_fin


kwant.plot(sys)

sys = sys.finalized()
 


ValueError: ignored

In [26]:
# -------------------------------------------------------
# Calculation
ky = 0.0
kx = 0.0

energies = np.arange(0.0, 5.0, 0.05)
transmission = []
num_prop = []
for energy in energies:
    smatrix = kwant.smatrix(sys, energy, [ky])
    transmission.append(smatrix.transmission(1, 0))
    num_prop.append(smatrix.num_propagating(0))
# -------------------------------------------------------
 
# Plot transmission and propagating modes
plt.plot(energies, transmission, '.')
plt.show()
 
plt.plot(energies, num_prop, '.')
plt.show()
 
# Plot wave function squared for the first mode for a specified energy and ky
#wf = kwant.solvers.default.wave_function(sys, energy=1.0, args=[1.0])
#kwant.plotter.map(sys, np.abs(wf(0)[0])**2, fig_size=(8,5));




    
def plot_conductance(syst, energies):
    # Compute conductance
    data = []
    for energy in energies:
        smatrix = kwant.smatrix(syst, energy)
        data.append(smatrix.transmission(1, 0))

    plt.figure()
    plt.plot(energies, data)
    plt.xlabel("energy [t]")
    plt.ylabel("conductance [e^2/h]")
    plt.show()

#sys, left_lead = make_system()
kwant.plot(sys)

def plot_density(sys,ener, it=-1):
    
    wf = kwant.solvers.default.wave_function(sys, energy=55.0, args=[0.0])

    kwant.plotter.map(sys, np.abs(wf(0)[0])**2, fig_size=(8,5))
    
    t=np.shape(wf(0))
    nwf=wf(0)[0]*0
    
    
    for i in range(t[0]//2+1):
        test=wf(0)[i]
        nwf+=test

    psi=abs(nwf)**2
    
    if it==-1:
        title="density"
    elif it>-1:
        title= "density"
        
    title2=title+".pdf"
    
    kwant.plotter.map(sys,psi,method='linear',vmin=0)

    J_0 = kwant.operator.Current(sys)
    c = J_0(nwf)

    kwant.plotter.current(sys, c)


    plt.close()


kwant.plot(sys)


kwant.plotter.bands(leadf,params=dict(t=1,k_y=0), show=False)
plt.xlabel("momentum [(lattice constant)^-1]")
plt.ylabel("energy [t]")
plt.show()
    
plot_density(sys, 2)
    #kwant.plotter.bands(left_lead)
plt.savefig('rysuneczek.pdf')
    
    
plt.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: KwantDeprecationWarning: The 'args' parameter has been deprecated since version 1.4 -- Instead, provide named parameters as a dictionary via 'params'.
  # Remove the CWD from sys.path while we load stuff.


TypeError: ignored